##### Import statements

In [1]:
import numpy as np
from tqdm import tqdm

##### The problem

In [2]:
data = """To be, or not to be, that is the question: Whether \
'tis nobler in the mind to suffer The slings and arrows of ou\
trageous fortune, Or to take arms against a sea of troubles A\
nd by opposing end them. To die—to sleep, No more; and by a s\
leep to say we end The heart-ache and the thousand natural sh\
ocks That flesh is heir to: 'tis a consummation Devoutly to b\
e wish'd. To die, to sleep; To sleep, perchance to dream—ay, \
there's the rub: For in that sleep of death what dreams may c\
ome, When we have shuffled off this mortal coil, Must give us\
 pause—there's the respect That makes calamity of so long lif\
e. For who would bear the whips and scorns of time, Th'oppres\
sor's wrong, the proud man's contumely, The pangs of dispriz'\
d love, the law's delay, The insolence of office, and the spu\
rns That patient merit of th'unworthy takes, When he himself \
might his quietus make""".lower()

chars = set( data)

data_size, char_size = len( data), len( chars)

print( f"Data Size: { data_size}, Char size: { char_size}")

char_to_idx = { c: i for i, c in enumerate( chars)}
idx_to_char = { i: c for i, c in enumerate( chars)}

train_X, train_y = data[ :-1], data[ 1:]

Data Size: 866, Char size: 32


##### Helper functions

In [3]:
def oneHotEncode( text):
	output = np.zeros( ( char_size, 1))
	output[ char_to_idx[ text]] = 1

	return output

def initWeight( input_size, output_size):
	return np.random.uniform( -1, 1, ( output_size, input_size)) * np.sqrt( 6 / ( input_size + output_size))

##### Activation functions

> when the derivative functions are called later in the code, they will be called on variables that have already had tanh of sigmoid applied. For this reason, the following functions are defined.

In [4]:
def sigmoid( input, derivative = False):
	if derivative:
		return input * ( 1 - input)
	
	return 1 / ( 1 + np.exp( -input))

def tanh( input, derivative = False):
	if derivative:
		return 1 - input ** 2
	
	return np.tanh( input)

def softmax( input):
	return np.exp( input) / np.sum( np.exp( input))

##### Long Short-Term Memory Network Class

In [5]:
class LSTM:
	def __init__( self, input_size, hidden_size, output_size):

		self.hidden_size = hidden_size

		# Forget Gate
		self.wf = initWeight( input_size, hidden_size)
		self.bf = np.zeros( ( hidden_size, 1))

		# Input Gate
		self.wi = initWeight( input_size, hidden_size)
		self.bi = np.zeros( ( hidden_size, 1))

		# Candidate Gate
		self.wc = initWeight( input_size, hidden_size)
		self.bc = np.zeros( ( hidden_size, 1))

		# Output Gate
		self.wo = initWeight( input_size, hidden_size)
		self.bo = np.zeros( ( hidden_size, 1))

		# Final Gate
		self.wy = initWeight( hidden_size, output_size)
		self.by = np.zeros( ( output_size, 1))

	# Reset Network Memeory
	def reset( self):
		self.concat_inputs = {}

		self.hidden_states = { -1: np.zeros( ( self.hidden_size, 1))}
		self.cell_states = { -1: np.zeros( ( self.hidden_size, 1))}

		self.activation_outputs = {}
		self.candidate_gates = {}
		self.output_gates = {}
		self.forget_gates = {}
		self.input_gates = {}
		self.outputs = {}

	# Forward Propagation
	def forward( self, inputs):
		self.reset()

		outputs = []

		for q in range( len( inputs)):
			self.concat_inputs[ q] = np.concatenate( ( self.hidden_states[ q - 1], inputs[ q]))

			self.forget_gates[ q] = sigmoid( np.dot( self.wf, self.concat_inputs[ q]) + self.bf)
			self.input_gates[ q] = sigmoid( np.dot( self.wi, self.concat_inputs[ q]) + self.bi)
			self.candidate_gates[ q] = tanh( np.dot( self.wc, self.concat_inputs[ q]) + self.bc)
			self.output_gates[ q] = sigmoid( np.dot( self.wo, self.concat_inputs[ q]) + self.bo)

			self.cell_states[ q] = self.forget_gates[ q] * self.cell_states[ q - 1] + self.input_gates[ q] * self.candidate_gates[ q]
			self.hidden_states[ q] = self.output_gates[ q] * tanh( self.cell_states[ q])

			outputs += [ np.dot( self.wy, self.hidden_states[ q]) + self.by]

		return outputs
	
	# Backward Propagation
	def backward( self, errors, inputs, learning_rate):
		d_wf, d_bf = 0, 0
		d_wi, d_bi = 0, 0
		d_wc, d_bc = 0, 0
		d_wo, d_bo = 0, 0
		d_wy, d_by = 0, 0

		dh_next, dc_next = np.zeros_like( self.hidden_states[ 0]), np.zeros_like( self.cell_states[ 0])
		for q in reversed( range( len( inputs))):

			error = errors[ q]

			# Final Gates Error
			d_wy += np.dot( error, self.hidden_states[ q].T)
			d_by += error

			# Hidden State Error
			d_hs = np.dot( self.wy.T, error) + dh_next

			# Output Gate Weights and Biases Errors
			d_o = tanh( self.cell_states[ q]) * d_hs * sigmoid( self.output_gates[ q], derivative=True)
			d_wo += np.dot( d_o, inputs[ q].T)
			d_bo += d_o

			# Cell State Error
			d_cs = tanh( tanh( self.cell_states[ q]), derivative=True) * self.output_gates[ q] * d_hs + dc_next

			# Forget Gate Weights and Biases Errors
			d_f = d_cs * self.cell_states[ q - 1] * sigmoid( self.forget_gates[ q], derivative=True)
			d_wf += np.dot( d_f, inputs[ q].T)
			d_bf += d_f

			# Input Gate Weights and Biases Error
			d_i = d_cs * self.candidate_gates[ q] * sigmoid( self.input_gates[ q], derivative=True)
			d_wi += np.dot( d_i, inputs[ q].T)
			d_bi += d_i

			# Candidate Gate Weights and Biases Errors
			d_c = d_cs * self.input_gates[ q] * tanh( self.candidate_gates[ q], derivative=True)
			d_wc += np.dot( d_c, inputs[ q].T)
			d_bc += d_c

			# Concatenated Input Error ( Sum of Error at Each Gate)
			d_z = np.dot( self.wf.T, d_f) + np.dot( self.wi.T, d_i) + np.dot( self.wc.T, d_c) + np.dot( self.wo.T, d_o)

			# Error of Hidden State and Cell State at Next TIme Step
			dh_next = d_z[ :self.hidden_size, :]
			dc_next = self.forget_gates[ q] * d_cs

		for d_ in ( d_wf, d_bf, d_wi, d_bi, d_wc, d_bc, d_wo, d_bo, d_wy, d_by):
			np.clip( d_, -1, 1, out=d_)

		self.wf += d_wf * learning_rate
		self.bf += d_bf * learning_rate

		self.wi += d_wi * learning_rate
		self.bi += d_bi * learning_rate

		self.wc += d_wc * learning_rate
		self.bc += d_bc * learning_rate

		self.wo += d_wo * learning_rate
		self.bo += d_bo * learning_rate

		self.wy += d_wy * learning_rate
		self.by += d_by * learning_rate

	def fit( self, inputs, labels, epochs, learning_rate):
		inputs = [ oneHotEncode( input) for input in inputs]

		for _ in tqdm( range( epochs)):
			predictions = self.forward( inputs)

			errors = []
			for q in range( len( predictions)):
				errors += [ -softmax( predictions[ q])]
				errors[ -1][ char_to_idx[ labels[ q]]] += 1

			self.backward( errors, self.concat_inputs, learning_rate)

	def predict( self, inputs, labels):
		hit = 0

		probabilities = self.forward( [ oneHotEncode( input) for input in inputs])

		output = ""

		for q in range( len( labels)):
			prediction = idx_to_char[ np.random.choice( [ *range( char_size)], p = softmax( probabilities[ q].reshape( -1)))]

			output += prediction

			if prediction == labels[ q]:
				hit += 1

		print( f"Ground Truth:\n\t{ labels}\n")
		print( f"Predictions:\n\t{ ''.join( output)}\n")

		print( f"Accuracy: { hit * 100 / len( inputs)}%")

In [6]:
hidden_size = 25

lstm = LSTM(
	input_size=char_size + hidden_size,
	hidden_size=hidden_size,
	output_size=char_size
)

lstm.fit(
	inputs=train_X,
	labels=train_y,
	epochs=1000,
	learning_rate=0.05
)

lstm.predict(
	inputs=train_X,
	labels=train_y
)

100%|██████████| 1000/1000 [01:56<00:00,  8.61it/s]


Ground Truth:
	o be, or not to be, that is the question: whether 'tis nobler in the mind to suffer the slings and arrows of outrageous fortune, or to take arms against a sea of troubles and by opposing end them. to die—to sleep, no more; and by a sleep to say we end the heart-ache and the thousand natural shocks that flesh is heir to: 'tis a consummation devoutly to be wish'd. to die, to sleep; to sleep, perchance to dream—ay, there's the rub: for in that sleep of death what dreams may come, when we have shuffled off this mortal coil, must give us pause—there's the respect that makes calamity of so long life. for who would bear the whips and scorns of time, th'oppressor's wrong, the proud man's contumely, the pangs of dispriz'd love, the law's delay, the insolence of office, and the spurns that patient merit of th'unworthy takes, when he himself might his quietus make

Predictions:
	o be, or not to be, that is the question: whether 'tis nobler in the mind to suffer the slings and arrow